Привет, путник!
Не знаю, зачем ты зашел в эту абсолютно хелповую тетрадь, но если уж зашел:

* Было замечено, что моделька плохо парсит объекты и аспекты в вопросах
* 2 варианта:
  ее смущает форма вопроса
  ее смущает слова, которые он не видела
  
* посмотреть, как оверлапится множество объектов для трейна и теста (ответ: очень хорошо, почти 100% перекрытие множеств именованных аспектов на трейне и тесте)
* добавть в датасет вопросы
* прикрутить елмо (не помогло)

## 1

In [1]:
!ls

ACQuA - CAM Aspects release 28.02.2019 - convertedratings.csv
CAM_dev.csv
CAM_qw_test.csv
CAM_test.csv
CAM_train.csv
explore_dataset.ipynb
run_command_line.ipynb


In [5]:
aspect_df = pd.read_csv('ACQuA - CAM Aspects release 28.02.2019 - convertedratings.csv')
aspect_df.head()

,OBJECT A,OBJECT B,ASPECT,ASPECT BELONGS TO,MOST FREQUENT RATING,CONFIDENCE,AMOUNT OF GOOD RATINGS,AMOUNT OF BAD RATINGS,SENTENCE EXAMPLE 1,SENTENCE EXAMPLE 2,SENTENCE EXAMPLE 3,SENTENCE EXAMPLE 4,SENTENCE EXAMPLE 5
0,golf,hockey,spot,golf,BAD,1.0,0,1,NaN,NaN,NaN,NaN,NaN
1,golf,hockey,greater,golf,BAD,1.0,0,1,"in fact, athletes in individual sports (i.e., ...","first, athletes in individual sports (i.e., go...",NaN,NaN,NaN
2,golf,hockey,prof,golf,BAD,1.0,0,1,"being in a massive hockey market, i can't thin...",NaN,NaN,NaN,NaN
3,golf,hockey,arm,golf,BAD,1.0,0,1,NaN,NaN,NaN,NaN,NaN
4,golf,hockey,mcphee,golf,BAD,1.0,0,1,NaN,NaN,NaN,NaN,NaN


In [5]:
import codecs

i = 0
with codecs.open('CAM_qw_test.csv', 'w', 'utf-8') as f:
    writer = csv.writer(f)
    for elem in zip(wq_df['word'].values, wq_df['tag'].values):
        print (elem)
        i = i+1
        writer.writerow(elem)

NameError: name 'codecs' is not defined

In [3]:
import pandas as pd

new_df = pd.read_csv("CAM_train.csv", names = ['word', 'tag'])
new_df_train = pd.read_csv("CAM_train_new.csv", names = ['word', 'tag'])

In [55]:
a = new_df_train['word'].values[43]

In [56]:
b = new_df['word'].values[43]

In [57]:
a == b

False

In [45]:
new_df_train['word'].values[-45:]

array(['football', 'or', 'volleyball', '?', nan, 'what', 'is', 'better',
       'football', 'or', 'lacrosse', '?', nan, 'what', 'is', 'better',
       'ds', 'or', 'psp', '?', nan, 'is', 'amazon', 'better', 'than',
       'itunes', '?', nan, 'is', 'basketball', 'worse', 'than', 'tennis',
       '?', nan, 'is', 'dell', 'worse', 'than', 'hewlett-packard', '?',
       nan, 'laptop', 'vs', 'phone'], dtype=object)

In [46]:
new_df['word'].values[40:45]

array(['and', 'seagate', '.', nan, 'amazon'], dtype=object)

In [10]:
list_ = []
for elem in new_df['tag'].values:
    if not(elem in list_):
        list_.append(elem)
list_

['O', 'B-OTHOBJ', 'B-ASPOBJ', nan, 'B-ASP', 'I-ASP', 'I-ASPOBJ', 'I-OTHOBJ']

In [6]:
list_of_object_pairs = []
for elem in zip(aspect_df['OBJECT A'].values, aspect_df['OBJECT B'].values):
    if elem not in list_of_object_pairs:
        list_of_object_pairs.append(elem)

In [14]:
train_names = []
test_names = []

for word, tag in zip(new_df['word'].values, new_df['tag'].values):
    if (tag == 'B-ASPOBJ' or tag == 'B-OTHOBJ'):
        if (word not in train_names):
            train_names.append(word)
            
for word, tag in zip(new_df_test['word'].values, new_df_test['tag'].values):
    if (tag == 'B-ASPOBJ' or tag == 'B-OTHOBJ'):
        if (word not in test_names):
            test_names.append(word)
            
train_names = sorted(train_names)
test_names = sorted(test_names)

In [15]:
print (len(train_names), len(test_names))

119 110


In [16]:
common = []
for elem in train_names:
    if elem in test_names:
        common.append(elem)
        
print (len(common))

110


** в общем, полный оверлаппинг **

*Создадим искуственный датасет на вопросах*

In [7]:
import random
import numpy as np
wq_df = pd.DataFrame(columns = ['word', 'tag'])

comparative_word = ['better', 'easier', 'faster', 'nicer', 'wiser','safer', 'worse', 'harder', 'slower', 'poorly']
for elem in list_of_object_pairs:
    qw_type = random.choice([0, 1, 0, 1, 2])
    if (qw_type == 0):
        wq_df = wq_df.append({'word':'is', 'tag':'O'}, ignore_index=True)
        wq_df =wq_df.append({'word':elem[0], 'tag':'B-ASPOBJ'}, ignore_index=True)
        
        asp = np.random.choice(comparative_word, 1, p=[0.55, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
        if (asp[0] != 'better'):
            wq_df = wq_df.append({'word':asp[0], 'tag':'B ASP'}, ignore_index=True)
        else:
            wq_df = wq_df.append({'word':asp[0], 'tag':'O'}, ignore_index=True)
        wq_df = wq_df.append({'word':'than', 'tag':'O'}, ignore_index=True)
        wq_df = wq_df.append({'word':elem[1], 'tag':'B-OTHOBJ'}, ignore_index=True)
        wq_df = wq_df.append({'word':'', 'tag':''}, ignore_index=True)
        
    elif (qw_type == 1):
        wq_df = wq_df.append({'word':'what', 'tag':'O'}, ignore_index=True)
        wq_df = wq_df.append({'word':'is', 'tag':'O'}, ignore_index=True)
        
        asp = np.random.choice(comparative_word, 1, p=[0.55, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
        if (asp[0] != 'better'):
            wq_df = wq_df.append({'word':asp[0], 'tag':'B ASP'}, ignore_index=True)
        else:
            wq_df = wq_df.append({'word':asp[0], 'tag':'O'}, ignore_index=True)
        
        wq_df = wq_df.append({'word':elem[0], 'tag':'B-ASPOBJ'}, ignore_index=True)
        wq_df = wq_df.append({'word':'or', 'tag':'O'}, ignore_index=True)

        wq_df = wq_df.append({'word':elem[1], 'tag':'B-OTHOBJ'}, ignore_index=True)
        wq_df = wq_df.append({'word':'', 'tag':''}, ignore_index=True)
        
    elif (qw_type == 2):
        wq_df = wq_df.append({'word':elem[0], 'tag':'B-ASPOBJ'}, ignore_index=True)
        wq_df = wq_df.append({'word':'vs', 'tag':'O'}, ignore_index=True)
        wq_df = wq_df.append({'word':elem[1], 'tag':'B-OTHOBJ'},ignore_index=True)
        wq_df = wq_df.append({'word':'', 'tag':''}, ignore_index=True)
    

In [88]:
wq_df.to_csv('CAM_qw_test.csv')

In [97]:
len(wq_df)

771

In [8]:
wq_df_to_train = wq_df[:300].reset_index(drop=True)
wq_df_to_test = wq_df[301:].reset_index(drop=True)

In [9]:
new_df = new_df.append(wq_df_to_train, ignore_index=True)

In [10]:
new_df.head(-45)

,word,tag
0,1tb,O
1,of,O
2,mechanical,O
3,storage,O
4,is,O
5,n't,O
6,bad,O
7,",",O
8,but,O
9,toshiba,B-OTHOBJ


In [11]:
new_df.to_csv('CAM_train_new.csv', header=False, index=False)

In [12]:
wq_df_to_test.to_csv('CAM_qw_new.csv', header=False, index=False)

In [37]:
import codecs
import csv

i = 0
with codecs.open('CAM_train_new.csv', 'w', 'utf-8') as f:
    writer = csv.writer(f)
    for elem in zip(new_df['word'].values, new_df['tag'].values):
        i = i+1
        writer.writerow(elem)

In [40]:
with codecs.open('CAM_qw_new.csv', 'w', 'utf-8') as f:
    writer = csv.writer(f)
    for elem in zip(wq_df_to_test['word'].values, wq_df_to_test['tag'].values):
        i = i+1
        writer.writerow(elem)

In [28]:
wq_df_to_test.reset_index(drop=True)

,word,tag
0,,
1,is,O
2,pasta,B-ASPOBJ
3,better,0
4,than,0
5,pizza,B-OTHOBJ
6,?,0
7,,
8,is,O
9,mysql,B-ASPOBJ


In [65]:
!pwd

/home/vika/NER_RNN/targer/data/NER/Asqua_CAM_aspects


In [66]:
import sys
sys.path
sys.path.append('/home/vika/NER_RNN/targer/')
from src.classes.utils import get_words_num


In [90]:
def read_data(fn, verbose=True, column_no=-1, splitter = ','):
        word_sequences = list()
        tag_sequences = list()
        with codecs.open(fn, 'r', 'utf-8') as f:
            lines = f.readlines()
        curr_words = list()
        curr_tags = list()
        print (len(lines))
        for k in range(len(lines)):
            #print (lines[k])
            line = lines[k].strip()
            if len(line) == 1 or line.startswith('-DOCSTART-'): # new sentence or new document
                if len(curr_words) > 0:
                    word_sequences.append(curr_words)
                    tag_sequences.append(curr_tags)
                    curr_words = list()
                    curr_tags = list()
                continue
            strings = line.split(',')
            #print (strings)
            word = strings[0]
            #print ("word", word)
            tag = strings[column_no] # be default, we take the last tag
            #print ("word", tag)
            curr_words.append(word)
            curr_tags.append(tag)
            if k == len(lines) - 1:
                word_sequences.append(curr_words)
                tag_sequences.append(curr_tags)
        if verbose:
            print('Loading from %s: %d samples, %d words.' % (fn, len(word_sequences), get_words_num(word_sequences)))
        return word_sequences, tag_sequences


In [91]:
a, b = read_data('CAM_train_new.csv')

77534
Loading from CAM_train_new.csv: 2661 samples, 74874 words.
